# build vocab
본 노트에서는 자연어처리 관련 딥러닝 모형학습을 위한 필수적인 전처리중 하나로 training corpus에 존재하는 token들의 집합인 **vocabulary**를 만들어 봅니다. vocabulary 구성 자체는 미리 제공하는 `model.utils` module에 있는 `Vocab` class를 활용해봅니다. `model.utils` module에 있는 `Tokenizer` class, `PadSequence` class를 같이 활용하여, 효율적인 전처리를 어떻게 할 수 있는 지 확인합니다.

### Setup

In [1]:
import sys
import pickle
import itertools
import pandas as pd
from pathlib import Path
from pprint import pprint
from typing import List
from collections import Counter
from model.utils import Vocab, Tokenizer, PadSequence

### Load dataset

In [2]:
data_dir = Path.cwd() / 'data'
list_of_dataset = list(data_dir.iterdir())
pprint(list_of_dataset)

[PosixPath('/root/Documents/archive/strnlp/exercise/data/morphs_vec.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/train.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/tokenizer.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/validation.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/vocab.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/test.txt')]


In [3]:
tr_dataset = pd.read_csv(list_of_dataset[1], sep='\t')
tr_dataset.head()

,document,label
0,애들 욕하지마라 지들은 뭐 그렇게 잘났나? 솔까 거기 나오는 귀여운 애들이 당신들보...,1
1,여전히 반복되고 있는 80년대 한국 멜로 영화의 유치함.,0
2,쉐임리스 스티브와 피오나가 손오공 부르마로 ㅋㅋㅋ,0
3,0점은 없나요?...,0
4,제발 시즌2 ㅜㅜ,1


### Split training corpus and count each tokens
앞선 `eda.ipynb` 노트에서 활용한 `Mecab` class의 instance의 멤버함수인 `morphs` 이용, **training corpus를 sequence of tokens의 형태로 변환하고, training corpus에서 token의 출현빈도를 계산합니다.**

In [4]:
# 문장을 형태소기준으로 보는 split_fn을 작성
from mecab import MeCab
split_morphs = MeCab().morphs

In [5]:
training_corpus = tr_dataset['document'].apply(lambda sen: split_morphs(sen)).tolist()
pprint(training_corpus[:5])

[['애',
  '들',
  '욕',
  '하',
  '지',
  '마',
  '라',
  '지',
  '들',
  '은',
  '뭐',
  '그렇게',
  '잘',
  '났',
  '나',
  '?',
  '솔',
  '까',
  '거기',
  '나오',
  '는',
  '귀여운',
  '애',
  '들',
  '이',
  '당신',
  '들',
  '보다',
  '훨',
  '낮',
  '다',
  '.'],
 ['여전히',
  '반복',
  '되',
  '고',
  '있',
  '는',
  '80',
  '년',
  '대',
  '한국',
  '멜',
  '로',
  '영화',
  '의',
  '유치',
  '함',
  '.'],
 ['쉐', '임', '리스', '스티브', '와', '피오나', '가', '손오공', '부르', '마', '로', 'ㅋㅋㅋ'],
 ['0', '점', '은', '없', '나요', '?', '...'],
 ['제발', '시즌', '2', 'ㅜㅜ']]


In [6]:
count_tokens = Counter(itertools.chain.from_iterable(training_corpus))
print(len(count_tokens))

48420


### Build vocab
`min_freq`를 설정하고, training corpus에서 출현빈도가 `min_freq` 미만인 token을 제외하여, `model.utils` module에 있는 `Vocab` class를 이용하여 Vocabulary를 구축합니다. `min_freq`보다 낮은 token들은 `unknown`으로 처리됩니다. 아래의 순서로 진행합니다.

1. `list_of_tokens`을 만듭니다. `list_of_tokens`은 `min_freq`보다 낮은 token들을 모아놓은 `list`입니다.
2. `Vocab` class의 instance인 `vocab`을 만듭니다. `list_of_tokens`를 parameter로 전달받습니다.

ps. tutorial에 사용되는 논문은 pretrained word vector를 사용하는 데, 원활한 진행을 위해서 제가 미리 준비해놓은 pretrained wordvector를 사용하도록 하겠습니다.

In [7]:
min_freq = 10
list_of_tokens = [token for token in count_tokens.keys() if count_tokens.get(token) >= min_freq]
list_of_tokens = sorted(list_of_tokens)
print(len(list_of_tokens))

8773


In [8]:
list_of_tokens = sorted(list_of_tokens)
vocab = Vocab(list_of_tokens=list_of_tokens, bos_token=None, eos_token=None, unknown_token_idx=0)

In [9]:
# tutorial 진행을 위해서 위에서 생성한 vocabulary의 token들의 embedding vector를 가져옵니다.
with open('data/morphs_vec.pkl', mode='rb') as io:
    morphs_vec = pickle.load(io)
print(morphs_vec)

[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [-0.25759   -0.014235  -1.2187    ... -0.097028   0.48018    0.28739  ]
 ...
 [ 0.022142  -0.19984   -0.68933   ...  0.057173   0.33876    0.063923 ]
 [-0.55427    0.12498   -1.0816    ...  0.0024414  0.34706    0.58391  ]
 [-0.56313    0.3403    -0.57685   ... -0.48403    0.38781   -0.49017  ]]


In [10]:
vocab.embedding = morphs_vec

### How to use `Vocab`
`list_of_tokens`로 생성한 `Vocab` class의 instance인 `vocab`은 아래와 같은 멤버들을 가지고 있습니다.

In [11]:
help(Vocab)

Help on class Vocab in module model.utils:

class Vocab(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, list_of_tokens=None, padding_token='<pad>', unknown_token='<unk>', bos_token='<bos>', eos_token='<eos>', reserved_tokens=None, unknown_token_idx=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  to_indices(self, tokens:Union[str, List[str]]) -> Union[int, List[int]]
 |  
 |  to_tokens(self, indices:Union[int, List[int]]) -> Union[str, List[str]]
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bos_token
 |  
 |  embedding
 |  
 |  eos_token
 |  
 |  idx_to_token
 |  
 |  padding_token
 |  
 |  token_to_idx
 |  
 |  unknown_token



In [12]:
# padding_token, unknown_token, eos_token, bos_token
print(vocab.padding_token)
print(vocab.unknown_token)
print(vocab.eos_token)
print(vocab.bos_token)

<pad>
<unk>
None
None


In [13]:
# token_to_idx
print(vocab.token_to_idx)

{'<unk>': 0, '<pad>': 1, '!': 2, '!!': 3, '!!!': 4, '!!!!': 5, '!!!!!': 6, '!!!!!!': 7, '!!!!!!!': 8, '!!!!!!!!': 9, '!!!!!!!!!': 10, '!!!!!!!!!!': 11, '!!!!!!!!!!!': 12, '!!!!!!!!!!!!!!!!!!!!!!!!!': 13, '!^^': 14, '!~~': 15, '!♥': 16, '"': 17, '""': 18, '"""': 19, '#': 20, '%': 21, '&': 22, "'": 23, '(': 24, ')': 25, '*': 26, '****': 27, '*^^*': 28, '+': 29, '+_+': 30, ',': 31, ',.': 32, ',..': 33, '-': 34, '-)': 35, '--': 36, '--;': 37, '-.-': 38, '-;': 39, '->': 40, '-_': 41, '-_-': 42, '-_-;': 43, '-_-;;': 44, '-_-;;;': 45, '.': 46, '.!': 47, '.!!': 48, '.!!!': 49, '."': 50, ".'": 51, '.(': 52, '.,': 53, '.,,': 54, '.-_-': 55, '.-_-;;': 56, '..': 57, '..!': 58, '..!!': 59, '..!!!': 60, '.."': 61, "..'": 62, '..(': 63, '..)': 64, '..-_-': 65, '...': 66, '....': 67, '.....': 68, '......': 69, '.......': 70, '........': 71, '.........': 72, '..........': 73, '...........': 74, '............': 75, '.............': 76, '..............': 77, '...............': 78, '................': 79,

In [14]:
# idx_to_token
print(vocab.idx_to_token)

{0: '<unk>', 1: '<pad>', 2: '!', 3: '!!', 4: '!!!', 5: '!!!!', 6: '!!!!!', 7: '!!!!!!', 8: '!!!!!!!', 9: '!!!!!!!!', 10: '!!!!!!!!!', 11: '!!!!!!!!!!', 12: '!!!!!!!!!!!', 13: '!!!!!!!!!!!!!!!!!!!!!!!!!', 14: '!^^', 15: '!~~', 16: '!♥', 17: '"', 18: '""', 19: '"""', 20: '#', 21: '%', 22: '&', 23: "'", 24: '(', 25: ')', 26: '*', 27: '****', 28: '*^^*', 29: '+', 30: '+_+', 31: ',', 32: ',.', 33: ',..', 34: '-', 35: '-)', 36: '--', 37: '--;', 38: '-.-', 39: '-;', 40: '->', 41: '-_', 42: '-_-', 43: '-_-;', 44: '-_-;;', 45: '-_-;;;', 46: '.', 47: '.!', 48: '.!!', 49: '.!!!', 50: '."', 51: ".'", 52: '.(', 53: '.,', 54: '.,,', 55: '.-_-', 56: '.-_-;;', 57: '..', 58: '..!', 59: '..!!', 60: '..!!!', 61: '.."', 62: "..'", 63: '..(', 64: '..)', 65: '..-_-', 66: '...', 67: '....', 68: '.....', 69: '......', 70: '.......', 71: '........', 72: '.........', 73: '..........', 74: '...........', 75: '............', 76: '.............', 77: '..............', 78: '...............', 79: '................',

In [15]:
# to_indices
example_sentence = tr_dataset['document'][0]
tokenized_sentence = split_morphs(example_sentence)
transformed_sentence = vocab.to_indices(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)

애들 욕하지마라 지들은 뭐 그렇게 잘났나? 솔까 거기 나오는 귀여운 애들이 당신들보다 훨 낮다.
['애', '들', '욕', '하', '지', '마', '라', '지', '들', '은', '뭐', '그렇게', '잘', '났', '나', '?', '솔', '까', '거기', '나오', '는', '귀여운', '애', '들', '이', '당신', '들', '보다', '훨', '낮', '다', '.']
[5219, 2455, 5877, 8244, 7157, 2854, 2636, 7157, 2455, 6159, 3347, 1197, 6585, 1667, 1539, 219, 4546, 1416, 693, 1576, 1901, 1132, 5219, 2455, 6219, 2095, 2455, 3764, 8682, 1673, 1953, 46]


In [16]:
# to_tokens
print(vocab.to_tokens(transformed_sentence))

['애', '들', '욕', '하', '지', '마', '라', '지', '들', '은', '뭐', '그렇게', '잘', '났', '나', '?', '솔', '까', '거기', '나오', '는', '귀여운', '애', '들', '이', '당신', '들', '보다', '훨', '낮', '다', '.']


### How to use `Tokenizer`
위의 `Vocab` class의 활용 형태를 보면 `split_fn`으로 활용하는 `split_morphs` function의 결과를 input을 기본적으로 받습니다. `Vocab` class의 instance와 `split_fn`으로 활용하는 `split_morphs` function을 input으로 받아 전처리를 해주는 `Tokenizer` class를 활용할 수 있습니다.

In [17]:
help(Tokenizer)

Help on class Tokenizer in module model.utils:

class Tokenizer(builtins.object)
 |  Tokenizer class
 |  
 |  Methods defined here:
 |  
 |  __init__(self, vocab:model.utils.Vocab, split_fn:Callable[[str], List[str]], pad_fn:Callable[[List[int]], List[int]]=None) -> None
 |      Instantiating Tokenizer class
 |      
 |      Args:
 |          vocab (model.utils.Vocab): the instance of model.utils.Vocab created from specific split_fn
 |          split_fn (Callable): a function that can act as a splitter
 |          pad_fn (Callable): a function that can act as a padder
 |  
 |  split(self, string:str) -> List[str]
 |  
 |  split_and_transform(self, string:str) -> List[int]
 |  
 |  transform(self, list_of_tokens:List[str]) -> List[int]
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object 

In [18]:
tokenizer = Tokenizer(vocab, split_morphs)

In [19]:
# split, transform, split_and_transform
example_sentence = tr_dataset['document'][1]
tokenized_sentence = tokenizer.split(example_sentence)
transformed_sentence = tokenizer.transform(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)

여전히 반복되고 있는 80년대 한국 멜로 영화의 유치함.
['여전히', '반복', '되', '고', '있', '는', '80', '년', '대', '한국', '멜', '로', '영화', '의', '유치', '함', '.']
[5547, 3548, 2370, 864, 6501, 1901, 189, 1766, 2111, 8307, 3109, 2750, 5657, 6198, 6092, 8374, 46]


In [20]:
print(tokenizer.split_and_transform(example_sentence))

[5547, 3548, 2370, 864, 6501, 1901, 189, 1766, 2111, 8307, 3109, 2750, 5657, 6198, 6092, 8374, 46]


`model.utils` module에 있는 `PadSequence`를 활용, `Tokenizer` class의 instance인 `tokenizer`에 padding 기능을 추가할 수 있습니다. 이 때 padding은 vocabulary에서 `<pad>`가 가리키고 있는 정수값을 활용합니다.

In [21]:
padding_value = vocab.to_indices(vocab.padding_token)
print(padding_value)

1


In [22]:
pad_sequence = PadSequence(length=32, pad_val=padding_value)

In [23]:
tokenizer = Tokenizer(vocab, split_morphs, pad_sequence)

In [24]:
# split, transform, split_and_transform
example_sentence = tr_dataset['document'][1]
tokenized_sentence = tokenizer.split(example_sentence)
transformed_sentence = tokenizer.transform(tokenized_sentence)
print(example_sentence)
print(tokenized_sentence)
print(transformed_sentence)

여전히 반복되고 있는 80년대 한국 멜로 영화의 유치함.
['여전히', '반복', '되', '고', '있', '는', '80', '년', '대', '한국', '멜', '로', '영화', '의', '유치', '함', '.']
[5547, 3548, 2370, 864, 6501, 1901, 189, 1766, 2111, 8307, 3109, 2750, 5657, 6198, 6092, 8374, 46, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### Save `vocab`

In [25]:
with open('data/vocab.pkl', mode='wb') as io:
    pickle.dump(vocab, io)